# Chapter 4
# Response surface methodology: Tuning continuous parameters 

In [ ]:
import random
import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
mpl.rcParams['figure.dpi']= 300

clr1 = "#333333"
clr2 = "#777777"
clr3 = "#AAAAAA"
clr4 = "#DDDDDD"
clrs = [clr1, clr2, clr3, clr4]
arrow_props = {'width':1, 'color': clr1,
                'headwidth': 5, 'headlength': 7}
font_size_2d = 7
chapter = 4

fig_dir = f"/Users/dsweet2/Desktop/Tuning Up/Chapter {chapter}/"
def save_fig_named(name):
    plt.tight_layout()
    for ext in ["eps", "png"]:
        plt.savefig(f"{fig_dir}/CH{chapter:02d}_{name}_sweet.{ext}")
        
def save_fig(fig_num):
    save_fig_named(f"F{fig_num:02d}")

In [ ]:
def horizontal_line(y0):
    c = plt.axis()
    plt.autoscale(False)
    plt.plot([c[0], c[1]], [y0, y0], '--', linewidth=1, color=clr3);

## 4.1	Tune a single continuous parameter

### 4.1.1	Design the experiment

#### Simulate a proprietary trading strategy

In [ ]:
def markout_profit(threshold):
    cost = 1
    pps = 1
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = pps*np.abs(signal) - cost + eps
    else:
        profit = 0
    return profit

In [ ]:
np.random.seed(17)
profit = np.array([markout_profit(threshold=1) for _ in range(10000)])
i = np.where(profit!=0)[0]
print (len(i), (len(profit)-len(i))/len(profit))
print(profit.mean(), profit.std())
print(profit[i].mean(), profit[i].std())
plt.hist(profit[i], 25);
save_fig(4)

#### CHOOSE THE PARAMETER VALUES

#### CONTINUOUS PARAMETERS IN RSM

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])
ax4.set_xticks([])
ax4.set_yticks([])

x = [1,2]
y = [1,2]
ax1.plot(x,y, 'o-', color=clr1);
ax1.axis([.5, 2.5, .5, 2.5])
ax1.set_xticks([1,2])
ax1.set_xticklabels('')
ax1.text(.6,2.2,'(a)')

x = [1,2]
y = [2,1]
ax2.plot(x,y, 'o-', color=clr1);
ax2.axis([.5, 2.5, .5, 2.5])
ax2.set_xticks([1,2])
ax2.set_xticklabels('')
ax2.text(.6,2.2,'(b)')

x = np.arange(1,2.001,.01)
x0 = 1.75
y = 2 - 2*(x-x0)**2
ax3.plot(x,y, '-',color=clr1);
ax3.plot([x[0], x[50], x[-1]], [y[0], y[50], y[-1]], 'o', color=clr1)
i=np.where(np.abs(x-x0)<.001)[0]
ax3.plot([x[i]], [y[i]], 'o', color=clr1, fillstyle='none')
ax3.axis([.5, 2.5, .5, 2.5])
ax3.set_xticks([1,1.5,2])
ax3.set_xticklabels('')
ax3.text(.6,2.2,'(c)')

x = np.arange(1,2.001,.01)
x0 = 1.25
y = 2 - 2*(x-x0)**2
ax4.plot(x,y, '-',color=clr1);
ax4.plot([x[0], x[50], x[-1]], [y[0], y[50], y[-1]], 'o', color=clr1)
i=np.where(np.abs(x-x0)<.001)[0]
ax4.plot([x[i]], [y[i]], 'o', color=clr1, fillstyle='none')
ax4.axis([.5, 2.5, .5, 2.5])
ax4.set_xticks([1,1.5,2])
ax4.set_xticklabels('')
ax4.text(.6,2.2,'(d)')

fig.text(.5, 0, 'threshold', ha='center')
fig.text(0, 0.5, 'profit', va='center', rotation='vertical')

save_fig(6)
plt.show()

#### DETERMINE THE NUMBER OF INDIVIDUAL MEASUREMENTS

In [ ]:
profit.mean()

In [ ]:
SD1 = 1.2
print ("SD1 =", SD1, profit.std())
PS = .03
print ("PS = ", PS, .2 * profit.mean())
N = (2.8 * SD1/PS)**2
print ("N =",N)
N = (3.08 * SD1/PS)**2
print ("N =",N)

### 4.1.2 Run and analyze the experiment

In [ ]:
def run_experiment(num_individual_measurements, thresholds):
    individual_measurements = {
     threshold: [] for threshold in thresholds
    }
    done = set()
    while True:
        threshold = np.random.choice(thresholds)
        profit = markout_profit(threshold)
        individual_measurements[threshold].append(profit)
        if (len(individual_measurements[threshold])
            >= num_individual_measurements):
            done.add(threshold)
        if len(done)==len(thresholds):
            break
    
    aggregate_measurements = []
    standard_errors = []
    for threshold in thresholds:
        ims = np.array(individual_measurements[threshold])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )
        
    return aggregate_measurements, standard_errors

In [ ]:
np.random.seed(17)
thresholds = np.array([1, 1.5, 2])
aggregate_measurements, standard_errors = run_experiment(15000, thresholds)

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=clr1, capsize=10);
plt.xlabel('threshold')
plt.ylabel('profit')

save_fig(8)
plt.show()

#### ITERATE TO FIND THE OPTIMUM

In [ ]:
np.random.seed(17)
thresholds = np.array([.5, 1, 1.5])
aggregate_measurements, standard_errors = run_experiment(15000, thresholds)
print (aggregate_measurements, standard_errors)

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=clr1, capsize=10);
plt.xlabel('threshold')
plt.ylabel('markout_profit')

save_fig(9)
plt.show()

**Sidebar: Linear regression and the normal equations**

In [ ]:
# univariate
x = np.array([1, 2, 3, 4])
y = np.array([.5, 1.1, 1.4, 2.1])
beta = (x*y).sum() / (x**2).sum()
print (beta)

In [ ]:
# multivariate
x = np.array([1, 2, 3, 4])
X = np.array([
  [1, x[0], x[0]**2],
  [1, x[1], x[1]**2],
  [1, x[2], x[2]**2],
  [1, x[3], x[3]**2]
])
y = np.array([.5, 1.1, 1.4, 1])
beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
print (beta)
# plt.plot(x,y ,'o--');

**End Sidebar: Linear regression and the normal equations**

In [ ]:
def linear_regression(thresholds, aggregate_measurements):
    x = thresholds
    y = aggregate_measurements
    X = np.array([np.ones(len(y)), x, x**2]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta

In [ ]:
beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

In [ ]:
def interpolate(thresholds, beta):
    xhat = np.arange(thresholds.min(), thresholds.max()+1e-6, .01)
    XHat = np.array([np.ones(len(xhat)), xhat, xhat**2]).T
    yhat = XHat @ beta
    return xhat, yhat

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=clr1, capsize=10);
xhat, yhat = interpolate(thresholds, beta)
plt.plot(xhat, yhat, '--', color=clr2)
plt.xlabel('threshold')
plt.ylabel('profit')

save_fig(10)
plt.show()

In [ ]:
beta = linear_regression(thresholds, aggregate_measurements)
print(beta)

In [ ]:
def optimize(thresholds, beta):
    xhat, yhat = interpolate(thresholds, beta)
    i = np.where(yhat==yhat.max())[0][0]
    return xhat[i], yhat[i]

In [ ]:
threshold_opt, estimated_max_profit = optimize(thresholds, beta)
print (threshold_opt, estimated_max_profit)

In [ ]:
plt.errorbar(thresholds, aggregate_measurements,
             yerr=standard_errors,
             fmt='o', color=clr1, capsize=10);
xhat, yhat = interpolate(thresholds, beta)
plt.plot(xhat, yhat, '--', color=clr2)
plt.plot(threshold_opt, estimated_max_profit, 'o', fillstyle='none', color=clr2)
# plt.errorbar(threshold_opt, estimated_max_profit, yerr=se_yhat, fmt='o', fillstyle='none', color=clr2)
plt.xlabel('threshold')
plt.ylabel('profit')

save_fig(11)
plt.show()

### 4.1.3	Validate the optimal parameter value

#### A SIMPLE VALIDATION MEASUREMENT

In [ ]:
estimated_max_profit, 

In [ ]:
np.random.seed(17)
aggregate_measurement, standard_error = run_experiment(15000, [threshold_opt])
print (aggregate_measurement[0]-2*standard_error[0], aggregate_measurement[0]+2*standard_error[0])

#### A MORE ROBUST VALIDATION MEASUREMENT

## 4.2	Tune two or more continuous parameters

In [ ]:
def markout_profit_2D(threshold, order_size):
    cost = 1
    pps = 1
    asc = .001*np.exp(2*order_size)
    signal = np.random.normal()
    eps = 2*np.random.normal()
    if (signal > threshold
        or signal < -threshold):
        profit = order_size*(pps*np.abs(signal) - cost + eps) - asc
    else:
        profit = 0
    return profit

**Sidebar: One factor at a time (OFAT)**

In [ ]:

def _interp_ofat(beta):
    x0_values = np.arange(-1, 1+1e-6, .01)
    x1_values = np.arange(-1, 1+1e-6, .01)
    x0hat_2d, x1hat_2d = np.meshgrid(x0_values, x1_values)
    x0hat = x0hat_2d.flatten()
    x1hat = x1hat_2d.flatten()
    XHat = np.array([np.ones(len(x0hat)), x0hat, x1hat, x0hat**2, x1hat**2, x0hat*x1hat]).T
    yhat = XHat @ np.array(beta)
    yhat_2d = np.reshape(yhat, (len(x1_values), len(x0_values)))
    return x0hat_2d, x1hat_2d, yhat_2d

def y_ofat(beta, x0, x1):
    x0hat, x1hat, yhat = _interp_ofat(beta)
    dx0 = np.abs(x0hat - x0)
    dx1 = np.abs(x1hat - x1)
    i = np.where( (dx0==dx0.min()) & (dx1==dx1.min()) )
    return yhat[i]

def _plot_1d_x0(ax, beta, x1hat0):
    x0hat, x1hat, yhat = _interp_ofat(beta)
    dx1 = np.abs(x1hat - x1hat0)
    i = np.where(dx1 == dx1.min())
    x0hat = x0hat[i[1]][0,:]
    x1hat = x1hat[i[1]][0,:]
    yhat = yhat[i[1]][0,:]
    ax.plot(
        x0hat,
        yhat,
        color=clr1
    )
    
    i = np.where(yhat == yhat.max())[0]
    return x0hat[i][0], yhat[i][0]
    
def _plot_1d_x1(ax, beta, x0hat0):
    x0hat, x1hat, yhat = _interp_ofat(beta)
    dx0 = np.abs(x0hat - x0hat0)
    i = np.where(dx0 == dx0.min())
    x0hat = x0hat[i[0]][:,0]
    x1hat = x1hat[i[0]][:,0]
    yhat = yhat[i[0]][:,0]    
    ax.plot(
        x1hat,
        yhat,
        color=clr1
    )
    i = np.where(yhat == yhat.max())[0]
    return x1hat[i][0], yhat[i][0]

def vline(ax, x0):
    c = ax.axis()
    ax.autoscale(False)
    ax.plot([x0, x0], [c[2], c[3]], '--', linewidth=1, color=clr3);    
    
beta = [10, .1, 1, -2.5, -5, 1]

print(y_ofat(beta, x0=-.4, x1=.8))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])
ax4.set_xticks([])
ax4.set_yticks([])

x1hat0_a = .8
ax1.plot([-1, 1], [x1hat0_a, x1hat0_a], '--', color=clr1)
ax1.axis([-1, 1, -1, 1])
ax1.set_yticks([x1hat0_a])
ax1.set_xticklabels('')
ax1.text(-.9, -.9, '(a)')
ax1.set_xlabel('x0')
ax1.set_ylabel('x1')

x0hat_opt_1d, yhat_opt_a = _plot_1d_x0(ax2, beta, x1hat0=x1hat0_a)
vline(ax2, x0hat_opt_1d)
ax1.plot([x0hat_opt_1d], [x1hat0_a], 'x', color=clr1, fillstyle='none')
ax2.set_yticks([yhat_opt_a])
ax2.set_yticklabels([f"$y^*={yhat_opt_a:.2f}$"], rotation=90)
ax2.set_xticks([x0hat_opt_1d])
ax2.set_xticklabels([f"$x0^*={x0hat_opt_1d:.2f}$"])
ax2.text(-1, 3.7, '(b)')

ax3.plot([-1, 1], [x1hat0_a, x1hat0_a], '--', color=clr1)
ax3.plot([x0hat_opt_1d, x0hat_opt_1d], [-1,1], '--', color=clr1)
ax3.axis([-1, 1, -1, 1])
ax3.set_yticks([x1hat0_a])
ax3.set_xticks([x0hat_opt_1d])
# ax3.set_xticklabels('')
ax3.set_xlabel('x0')
ax3.set_ylabel('x1')
ax3.text(-.9, -.9, '(c)')

x1hat_opt_1d, yhat_opt_1d = _plot_1d_x1(ax4, beta, x0hat0=x0hat_opt_1d)
vline(ax4, x1hat_opt_1d)
ax3.plot([x0hat_opt_1d], [x1hat_opt_1d], 'x', color=clr1, fillstyle='none')
ax4.set_xticklabels('')
ax4.set_yticks([yhat_opt_1d])
ax4.set_yticklabels([f"$y^*={yhat_opt_1d:.2f}$"], rotation=90)
ax4.set_xticks([x1hat_opt_1d])
ax4.set_xticklabels([f"$x1^*={x1hat_opt_1d:.2f}$"])
ax4.text(-1, 6.7, '(d)')

save_fig_named("ofat_1")

plt.show()

In [ ]:
def plot_ofat(beta):
    x0hat, x1hat, yhat = _interp_ofat(beta)
    plt.contourf(x0hat, x1hat, yhat, alpha=.5, cmap="Greys")
    
    i = np.where(yhat == yhat.max())
    return x0hat[i][0], x1hat[i][0], yhat[i][0]

x0hat_opt, x1hat_opt, yhat_opt = plot_ofat(beta)
vline(plt, x0hat_opt)
horizontal_line(x1hat_opt)
plt.plot(x0hat_opt, x1hat_opt, 'o', fillstyle='none', color=clr1)
plt.xticks([x0hat_opt], labels=[f"$x0^* = {x0hat_opt:.2f}$"])
plt.yticks([x1hat_opt], labels=[f"$x1^* = {x1hat_opt:.2f}$"], rotation=90, va='center')


plt.plot([-1, 1], [x1hat0_a, x1hat0_a], '--', color=clr1)
plt.plot([x0hat_opt_1d, x0hat_opt_1d], [-1,1], '--', color=clr1)
plt.plot([x0hat_opt_1d], [x1hat_opt_1d], 'x', color=clr1)

plt.text(x0hat_opt+.05, x1hat_opt+.07, "$y^{**}="+f"{yhat_opt:.1f}$")
plt.text(x0hat_opt_1d-.25, x1hat_opt_1d-.07, "$y^{*}="+f"{yhat_opt_1d:.1f}$")

save_fig_named("ofat_2")

plt.show();

**End Sidebar: One factor at a time (OFAT)**

## 4.2.1	Design the experiment

In [ ]:
thresholds = [1, 1.5, 2]
order_sizes = [1, 2, 3]
axes = [.95, 2.05, .9, 3.1]

fig, ((ax1, ax2, ax3)) = plt.subplots(1, 3)
ax1.set_xticks([])
ax1.set_yticks([])
ax2.set_xticks([])
ax2.set_yticks([])
ax3.set_xticks([])
ax3.set_yticks([])

ax1.plot(thresholds, [2, 2, 2], 'o', color=clr1);
ax1.axis(axes)
ax1.set_aspect(.5)
ax1.set_xticks(thresholds)
ax1.text(1.5, .3, 'threshold', ha='center')
ax1.text(1, 2.8, '(a)')

ax2.plot([1.5, 1.5, 1.5], order_sizes, 'o', color=clr1);
ax2.axis(axes)
ax2.set_aspect(.5)
ax2.set_yticks(order_sizes)
ax2.set_xticks([-10])

ax2.text(1.5, .3, 'threshold', ha='center')
ax2.text(1, 2.8, '(b)')

ax3.plot(thresholds, [2, 2, 2], 'o', color=clr1);
ax3.plot([1.5, 1.5, 1.5], order_sizes, 'o', color=clr1);
ax3.axis(axes)
ax3.set_aspect(.5)
ax3.set_xticks(thresholds)
ax3.set_yticks(order_sizes)
ax3.text(1.5, .3, 'threshold', ha='center')
ax3.text(1, 2.8, '(c)')

fig.text(0.0, 0.5, 'order_size', va='center', rotation='vertical')

save_fig(14)
plt.show()

In [ ]:
def design_ccd(thresholds, order_sizes):
    parameters = [
        (threshold, order_size)
        for threshold in thresholds
        for order_size in order_sizes
    ]
    return parameters

In [ ]:
parameters = design_ccd(thresholds=[1, 1.5, 2], order_sizes=[1, 2, 3])
print (parameters)

In [ ]:
pp = np.array(parameters)
thresholds = pp[:,0]
order_sizes = pp[:,1]
plt.plot(thresholds, order_sizes, 'o', color=clr1)
plt.xlabel('threshold')
plt.ylabel('order size')
save_fig(15)
plt.show()

### 4.2.2	Run, analyze, and validate the experiment

In [ ]:
np.random.seed(17)
profit = np.array([markout_profit_2D(threshold=1, order_size=1) for _ in range(10000)])
i = np.where(profit!=0)[0]
print (len(i), (len(profit)-len(i))/len(profit))
print(profit.mean(), profit.std())

#### RUN THE EXPERIMENT

In [ ]:
def run_experiment_2D(num_individual_measurements, parameters):
    individual_measurements = {
      parameter: [] for parameter in parameters
    }
    done = set()
    while True:
        parameter = parameters[np.random.choice(len(parameters))]
        threshold, order_size = parameter
        profit = markout_profit_2D(threshold, order_size)
        individual_measurements[parameter].append(profit)
        if (len(individual_measurements[parameter])
            >= num_individual_measurements):
            done.add(parameter)
        if len(done) == len(individual_measurements):
            break
    
    aggregate_measurements = []
    standard_errors = []
    for parameter in parameters:
        ims = np.array(individual_measurements[parameter])
        aggregate_measurements.append( ims.mean() )
        standard_errors.append( ims.std()/np.sqrt(len(ims)) )
        
    return aggregate_measurements, standard_errors

In [ ]:
np.random.seed(17)
parameters = design_ccd(thresholds=[1, 1.5, 2], order_sizes=[1, 2, 3])
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)

In [ ]:
def _plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors):
    n = np.arange(len(parameters))
    plt.errorbar(n, aggregate_measurements,
                 yerr=standard_errors,
                 fmt='o', color=clr1, capsize=10);
    plt.ylabel('markout profit', fontsize=font_size_2d)
    plt.yticks(fontsize=font_size_2d)
    plt.xticks(
        ticks=list(range(len(parameters))),
        labels=[f"th={p[0]:.1f}\nos={p[1]:.1f}" for p in parameters],
        fontsize=font_size_2d
    )


In [ ]:
_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
save_fig(17)

#### ANALYZE THE EXPERIMENT

In [ ]:
def linear_regression_2D(parameters, aggregate_measurements):
    parameters = np.array(parameters)
    x0 = parameters[:,0]
    x1 = parameters[:,1]
    y = np.array(aggregate_measurements)
    X = np.array([np.ones(len(y)), x0, x1, x0**2, x1**2, x0*x1]).T
    beta = np.linalg.inv(X.T @ X) @ (X.T @ y)
    return beta

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
print (beta)

In [ ]:
def interpolate_2D(parameters, beta):
    parameters = np.array(parameters)
    x0_values = np.arange(parameters[:,0].min(), parameters[:,0].max()+1e-6, .01)
    x1_values = np.arange(parameters[:,1].min(), parameters[:,1].max()+1e-6, .01)
    x0hat_2d, x1hat_2d = np.meshgrid(x0_values, x1_values)
    x0hat = x0hat_2d.flatten()
    x1hat = x1hat_2d.flatten()
    XHat = np.array([np.ones(len(x0hat)), x0hat, x1hat, x0hat**2, x1hat**2, x0hat*x1hat]).T
    yhat = XHat @ beta
    yhat_2d = np.reshape(yhat, (len(x1_values), len(x0_values)))
    return x0hat_2d, x1hat_2d, yhat_2d

In [ ]:
def _plot_interpolation(parameters, aggregate_measurements, beta, parameter_opt=None):
    parameters = np.array(parameters)
    thresholds = parameters[:,0]
    order_sizes = parameters[:,1]
    x0hat, x1hat, yhat = interpolate_2D(parameters, beta)
    fig = plt.figure()
    plt.contourf(x0hat, x1hat, yhat, alpha=.5, cmap="Greys")
    plt.plot(thresholds, order_sizes, 'X', color=clr1) #, aggregate_measurements, 'o', color=clr1)
    plt.colorbar()
    plt.title('markout_profit')
    
    if parameter_opt is not None:
        threshold_opt = parameter_opt[0]
        order_size_opt = parameter_opt[1]
        plt.plot(threshold_opt, order_size_opt, 'o', color=clr1, markersize=20, fillstyle='none')
    
    plt.xlabel('threshold')
    plt.ylabel('order size')

In [ ]:
_plot_interpolation(parameters, aggregate_measurements, beta)
save_fig(18)
plt.show()

In [ ]:
def optimize_2D(parameters, beta):
    x0hat, x1hat, yhat = interpolate_2D(parameters, beta)
    i = np.where(yhat==yhat.max())
    return x0hat[i][0], x1hat[i][0], yhat[i][0]

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
threshold_opt, order_size_opt, estimated_max_profit = optimize_2D(parameters, beta)
print (threshold_opt, order_size_opt, estimated_max_profit)

In [ ]:
parameters = design_ccd(thresholds=[.5,  1,  1.5], order_sizes=[2.5, 3, 3.5])
np.random.seed(17)
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)
aggregate_measurements_prev, standard_errors_prev = aggregate_measurements, standard_errors

_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
axis_prev = plt.axis()
save_fig(19)

In [ ]:
parameters = design_ccd(thresholds=[.75,  1, 1.25], order_sizes=[2.75, 3, 3.25])
np.random.seed(17)
aggregate_measurements, standard_errors = run_experiment_2D(15000, parameters)

_plot_aggregate_measurements(parameters, aggregate_measurements, standard_errors)
c = plt.axis()
plt.axis([c[0], c[1], axis_prev[2], axis_prev[3]])
save_fig(20)

In [ ]:
beta = linear_regression_2D(parameters, aggregate_measurements)
threshold_opt, order_size_opt, estimated_max_profit = optimize_2D(parameters, beta)
print (threshold_opt, order_size_opt, estimated_max_profit)

ax = _plot_interpolation(parameters, aggregate_measurements, beta, (threshold_opt, order_size_opt))
save_fig(21)
plt.show()

#### VALIDATE THE INTERPOLATION ESTIMATE

In [ ]:
np.random.seed(17)
aggregate_measurement, standard_error = run_experiment_2D(
    num_individual_measurements=15000,
    parameters=[(threshold_opt, order_size_opt)]
)

In [ ]:
print (aggregate_measurement, standard_error)
print(aggregate_measurement[0] - 2*standard_error[0], aggregate_measurement[0] + 2*standard_error[0])